# A Survey of Transformers

Transformer는 NLP, CV, audio processing과 같은 많은 분야에서 좋은 성과를 달성했습니다. 현재 까지 매우 다양한 Transformer의 변형(a.k.a. X-formers)이 제안되었지만 이러한 변형에 대한 체계적이고 포괄적인 리뷰는 아직 없습니다. 이 survey에서는 다양한 X-formers에 대한 포괄적인 리뷰를 제공합니다.

## INTRODUTION

Transformer는 원래 기계 번역을위한 sequence-to-sequence 모델로 제안되었습니다.  
이후 연구에 따르면 transformer 기반의 pre-trained model(PTM)은 다양한 작업에서 SOTA를 달성했다.  
이러한 성공으로 인해 지난 몇 년 동안 다양한 Transformer 변형 (일명 X-formers)이 제안되었습니다.  
이 X-formers는 다양한 관점에서 바닐라 트랜스포머를 개선합니다.

1. **Model Efficiency**  
Transformer적용의 주요 과제는 self-attention의 긴 문장 처리에 대한 복잡한 계산, 메모리 사용량 입니다.  
개선 방법에는 light weight attention, Divide-and-conquer method등입니다.
2. **Model Deneralization**  
Transformer는 유연한 아키텍처이고 입력 데이터의 구조적 편향에 대한 가정을 거의 하지 않기 때문에 적은 데이터로는 훈련이 힘듭니다.
개선 방법에는 structural bias, 정규화, 많이 데이터의 사전 훈련등 입니다.
3. **Model Adaptation**  
특정 task, application에 맞게 조정하는 것을 목표로 합니다.

위 3가지 관점을 기반으로 기존의 많은 X-former들의 문제를 해결가능합니다. 따라서 기존의 다양한 변형들을 분류하고 주로 바닐라 Transformer를 개선하는 방법에 따라 아키텍처를 수정, 사전 훈련, application에 따라 새로운 분류를 제안하는 것이 체계적입니다.

**주로 일반적인 아키텍처 변형에 초점을 맞추고, 사전 훈련, application에 대한 변형에 대하 간단히 논의합니다.**

## BACKGROUND
### Vanilla Transformer
처음 제안된 Transformer에 대한 설명입니다.
간단히 요약하고 넘어가겠습니다.

Transformer의 전체 아키텍처는 아래와 같습니다.

Fig. 1. 

#### Attention Modules
Transformer는 모델에 Query-Key-Value(Q, K, V)로 어텐션 메커니즘을 적용합니다.
scaled dot-product attention은 아래와 같습니다.

\begin{equation}
Attention(Q, K, V) = softmax\left(\frac{QK^T}{\sqrt{D_k}}\right)V = AV 
\end{equation}

여기서 Dk로 나누는 것은 softmax function의 gradient vanishing 문제를 완화해줍니다. 

간단히 싱글 어텐션을 적용하기보다 Transformer는 multi-head attention(이하 MHA)을 사용합니다. 여기서 Q, K, V는 $D_q, D_k, D_v$차원으로 projection됩니다.


\begin{equation}
MultiHeadAttn(Q, K, V) = Concat(head_1, \cdots, head_H)W^O,
\end{equation}


\begin{equation}
where\  head_i = Attention(QW_i^Q, KW_i^K, VW_i^V).
\end{equation}


Transformer에는 3가지 attention type이 존재합니다.
1. **self-attention.**  
트렌스포머 인코더에서 Eq.(2) 는 $Q = K = V = X$로 설정됩니다. 여기서 $X$는 이전 레이어의 출력입니다.
2. **Masked Self-attention**  
디코더에서 Q는 이후의 K에 attention할수 없도록 합니다. 
3. **Cross-attention**  
Q는 디코더에서 K, V는 인코더에서 옵니다.

#### Position-wise FFN
a fully connected feed-forward module입니다.


\begin{equation}
FFN(H') = ReLU(H'W^1 + b)W^2 + b^2, 
\end{equation}

여기서 H는 이전 레이어의 출력입니다.

#### Resifual Connection and Normalization
각 모듈에 Layer Normalization이후 residual connection을 적용합니다.


\begin{equation}
H' = LayerNorm(SelfAttention(X) + X)
\end{equation}


\begin{equation}
H = LayerNorm(FFN(H') + H')
\end{equation}



### Model Usage
일반적으로 트렌스포머는 3가지 방법으로 사용됩니다.

1. **Encoder-Decoder**  
full Transformer architecture입니다. 일반적으로 seq-to-seq모델링에 사용됩니다.
2. **Encoder only**  
출력값을 입력문장의 representatino이므로, classification, labeling에 주로 사용됩니다.
3. **Decoder only**  
cross attention module이 제거됩니다. 언어, 생성 모델에 사용됩니다.

### Model Analysis

연산량과 필요 파라미터를 설명하기 위해서 트렌스포머의 두가지 속성을 아래 표로 분석합니다.  
여기서 D는 hidden dimention, T는 입력의 길이입니다. FFN의 중간 차원은 4D이며 Q, K, V의 차원은 D/H입니다.

table1.

입력이 짧은 경우 D가 복잡성을 지배합니다. 즉 병목현상은 FFN에서 일어납니다. 반대는 self-attention에서 일어납니다.(입력이 너무 길면 T by T matrix를 만들어야 하기 때문에 실험 불가능함)  
즉, Transformer(이하 Trm)의 효율성은 긴문장의 호완성 + FFN의 효율성으로 이어집니다.

### Comparing Transformer to Other Network Types
#### Analysis of Self-Attention
Trm의 중심 부분인 self-attention은 가변 길이의 input을 처리 할수 있는 유연한 메커니즘과 홤께 제공됩니다. 이는 가중치가 입력의 관계에서 동적으로 생성되는 fully connected layer로 이해 될수 있습니다. Table2는 layer의 타입별 비교를 나타냅니다.

table2.

self-attention의 장점은 다음과 같습니다.
1. Maximum path를 가지므로, 장거리 종속성을 가진 모델링에 적합합니다. fully connected layer에 비해 가변길이 입력을 처리하는데 더 효율적이고 유연합니다.
2. CNN은 글로벌 수용력을 갖기 위해 layer을 쌓아야 하지만, 일정한 최대 길이를 사용하면 일정한 수의 layer로 장거리 종속석을 가질수 있습니다.
3. recurrent layer보다 장거리 모델링에서 self-attention을 더 병렬화 하고 잘 관리 할 수 있습니다.

####  In Terms of Inductive Bias
Trm은 ConV, recurrent와 비교됩니다. Conv는 공유 커널로, 대칭 불변성, 지역성 편향을 부과합니다. recurrent Net은 Markovian 구조로 시간 불변성과 지역성의 유도 편향을 부과합니다. Trm은 데이터의 구조정보에 대한 가정이 거의 없습니다. 이는 Trm을 보편적이고 유연한 아케텍처로 만듦니다. 부작용으로는 구조적 편향이 없기 때문에, Trm은 적은 데이터에 대해 오버피팅되는 경향이 있습니다.

또다른 유현은 GNN의 message passing입니다. Trm은 입력 데이터가 어떻게 구조화 되는지에 대한 사전 지식을 도입하지 않기 때문에 Trm의 메시지 전달 프로세스는 contents에 대한 유사성 측정에만 의존합니다.

## TAXONOMY OF TRANSFORMERS
지금 까지 아키텍처 수정 유형, 사전 훈련 방법, 응용 프로그램의 3가지 관점에서 바닐라 Trm을 기반으로 한 다양한 모델이 제안되어 왔습니다. 아래 그림은 Trm변형의 카테고리화 입니다.

Fig. 2. 

## ATTENTION
self attention은 Trm에서 중요한 역할을 하지만 실제 적용에서 두가지 어려움이 있습니다.

1. **Conplexity**  
Sec. 2.3에서 서술한것 처럼 긴문장에 대한 병목현상이 있습니다.
2. **Structuual prior**  
입력에 대한 구조적인 가정이 없습니다. 심지어 순서정보도 학습으로 배워야 합니다. 이는 적은 데이터에서의 오버피팅이 매우 쉽습니다.

위 어려움에 대한 개선은 여러 방향으로 나눠집니다.
1. **Sparse Attention**  
self attention에 sparsity bias을 도입하여 복잡성을 줄입니다.
2. **Linearized Attention**  
kernel feature map으로 attetion matrix를 분리합니다. 그런 다음 역순으로 계산하여 복잡성을 선형으로 줄입니다.
3. **Prototype and Memory Compression**  
Q, K-V 메모리 쌍의 수를 줄여서 attetion matrix의 크기를 줄입니다.
4. **Low-rank Self-Attention**  
이는 self-attention의 low rank 속성을 활용합니다.
5. **Attention with Prior**  
attention를 prior 분포로 보완하거나 대체하는 방법을 탐색합니다.
6. **Improved Multi-Head Mechanism**  
다양한 Multi-Head 메커니즘을 탐구합니다.

###  Sparse Attention
기본 self-attention은 모든 토큰을 다른 모든 토큰에 attention해야합니다. 하지만 학습된 model의 경우 attention matrix가 대부분의 point에서 희소합니다 따라서 구조적 편향을 통합하여 각 쿼리가 처리하는 Q-K쌍의 수를 제한 함으로써 계산 복잡성을 줄입니다. 이 제한 하에서 미리 정의된 패턴에 따라 Q-K 쌍의 유사성 점수를 계산합니다.

\begin{equation}
\hat{A}_{ij}= 
\begin{cases}
q_ik_j^T, & \mbox{if token }i \mbox{ attends to token } j \\
- \infty, & \mbox{if token }i \mbox{ does not attend to token } j \\
\end{cases}
\end{equation}

다른 관점으로는 바닐라 Trm은 완전 이분 그래프로, 희소 att는 일부 연결이 제거된 그래프로 볼수 있습니다.  
Sparse Attention을 결정하는 메트릭을 기반으로 위치 기반, 콘텐츠 기반 Sparse Attention이라는 두가지로 분류합니다.

####  Position-based Sparse Attention
A가 일부 미리 결정된 패턴에 따라 제한됩니다.

##### Atomic Sparse Attention.
미리 정의된 패턴들입니다.(데이터의 구조적 편향을 반영하는 방식으로 보입니다)   

Fig. 4.

1. **Global Attention**  
장거리 종속성 저하를 완화 하기 위해 일부 전역노드를 추가 하였습니다.(그림에서 상위 2개)
2. **Band Attention(a.k.a sliding window attention or local attention)**   
대부분의 데이터는 강력한 지역 속성을 갖기 때문에 각 쿼리를 인접 노드로 제한 하는것이 당연합니다.
3. **Dilated Attention**  
Dilated CNN처럼 dand attention의 수용력을 늘리는 시도 입니다.
4. **Random Attention**  
non-local interaction능력을 증가 시킵니다.
5. **Block Local Attention**  
Q가 겹치지 않는 메모리 블록으로 연결됩니다.

##### Compound Sparse Attention

FIg. 5.

+ star-transformer는 너비가 3인 global + band를 사용합니다. 인접하지 않은노드는 공유 전역 노드를 통해 연결되고, 인접 노드는 직접 연결 됩니다. 이를 그래프로 그래면 별 모양 그래프를 형성합니다.  
+ long-former는 band + internel global를 사용합니다. 글로벌 노드는 분류를 위해 \[CLS\]이 선택되고, QnA task를 위해서는 모든 질문 토큰으로 선택 됩니다. 추가로 수용력을 높히기 위해 일부는 dilated를 사용합니다.
+ Extended Transformer Construction(ETC)는 external global + band를 사용합니다. 또한 구조화 된 입력을 처리하고 사전 훈련을 위해 Contrastive Predictive Coding(CPC)을 적용하는 마스킹 메커니즘을 사용합니다.
+ bigbird는 global + random을 사용하여 대략적인 관심사를 표착 합니다.
+ Sparse Transformer는 서로다른 유형의 데이터에 대해 다른 패턴을 설계하였습니다.
    1. 이미지(주기적인 구조)는 band + strided
    2. 텍스트(주기 X)는 global(fixed position) + block


##### Extended Sparse Attention
위의 패턴 말고도, 일부 기존 연구에서 특정 데이터 유형에 대한 확장된 희소 패턴을 탐색합니다.

Fig. 6. 


text(BP-transformer) : 모든 토큰이 leaf node, 내부노드가 많은 토큰을 포합하는 span node인 바이너리 트리를 구성합니다.
이 그래프의 edge는 각 leaf node가 인접 leaf node와 더 먼 거리의 토큰을 포함하는 상위 범위 node에 연결됩니다. 이 접근법은 global노드가 계층적으로 구성되고 토큰 쌍이 이진 트리의 경로와 연결되는 global의 확장입니다.

image(image-transformer) : 두가지 유형이 있습니다.

#### Content-based Sparse Attention
입력 내용을 기반으로 그래프를 생성합니다. - 입력에 따라 연결이 조절됩니다.  
Content-based Sparse graph를 구성하는 간단한 방법은 주어지 Q에 대해 큰 유사성 점수를 가질 가능성이 높은 K를 선책하는 것 입니다. 희소 그래프를 효율적으로 구성하기 위해 MIPS문제를 되풀이 할수 있습니다. -> 모든 Q, K 의 product를 계산하지 않고 Q를 사용하여 최대 내적을 가진 K를 찾습니다.  

Routing transformer는 K-means 클러스터링을 해서 안에 있는 Key와 연산합니다. 클러스터의 중심 $\mu_k$는 훈련중에 moving average로 이동됩니다.

\begin{equation}
\tilde{\mu} \gets \lambda\tilde{\mu} + (1 - \lambda)\left( \sum_{i:\mu(q_i)=\mu}q_i + \sum_{j:\mu(k_j)=\mu}k_j \right)
\end{equation}

\begin{equation}
c_{\mu} \gets  \lambda c_{\mu} + (1 - \lambda) \left| \mu \right|
\end{equation}

\begin{equation}
\mu \gets \frac{\tilde{\mu}}{c_{\mu}}
\end{equation}


$\mathcal{P_i}$를 𝑖 번째 쿼리가 처리하는 키 인덱스 집합이라고 하면 Routing Transformer은 아래와 같이 정의 합니다.


\begin{equation}
\mathcal{P_i} = \{j : \mu (q_i) = \mu(k_j)\}
\end{equation}



Reformer는 locality-sensitive hashing (LSH)을 사용하여 각 Q에 대한 K-V쌍을 선택합니다. LSH attention을 통해 각 토큰을 동일한 해싱 버킷내의 토큰에만 attention을 합니다. 기본 아이디어는 LSH함수를 사용하여 Q, K를 여러 버킷으로 해시하여 유사한 항목이 높은 확률로 동일한 버킷에 포함되는것입니다. 특히 LSH 함수에 대해 random 행렬 방법을 사용합니다.  
b를 버킷의 수라고 할때, 크기가 $[D_km, b/2]$인 랜덤 행렬 $R$이 주어지면 LSH attention을 통해 𝑖 번째 Q는 인덱스가있는 K-V 쌍에만 집중할 수 있습니다.

\begin{equation}
h(x) = arg\  max([xR; -xR])
\end{equation}

\begin{equation}
\mathcal{P_i} = \{j : h (q_i) = h(k_j)\}
\end{equation}

### Linearized Attention

식(1)은 입력 길이 T에 대하여 quadratic 복잡성을 가집니다. 아래 그림에 표현된것처럼 Q, K의 소프트 맥스는 Q',k'의 연산으로 풀릴수 있습니다. 따라서 Q', K', V를 역순으로 연산하여 복잡성을 선형을 따르도록 합니다.

$\hat{A}$를 Q, K의 exp한 값의 합이라 두고 $D^{-1} = diag(\hat{A} 1_T^T)^{-1}$라 두면 $Z = D^{-1}\hat{A}V$는 원래의 연산과 같습니다.  
즉 $D^T\hat{A} = softmax(QK^T)$입니다.

선형화 된 attention을 위해 벡터 형식으로 공식을 도출합니다. 여기선 일반적인 형태의 attention을 고려합니다.

\begin{equation}
z_i = \sum_j\frac{sim(q_i, k_{j})}{\sum_{j'}sim(q_i, k_{j'})}v_j,
\end{equation}

여기서 sim은 입력간 유사성을 측정하는 스코어링 함수 입니다. 바닐라 Trm에서는 내적 exp함수입니다. 유사성 함수로 커널 $\mathcal{K}(x, y) = \phi(x)\phi(y)^T$를 사용하는것은 자연스러운 선택입니다.

\begin{equation}
z_i = \sum_j\frac{\phi(q_i)\phi(k_j)^T}{\sum_{j'}\phi(q_i)\phi(k_j')^T}v_j,
\end{equation}

\begin{equation}
= \frac{\phi(q_i) \color{Blue}{\sum_j \phi(k_j)^T \otimes v_j}}{\phi(q_i) \color{Blue}{\sum_{j'}\phi(k_j')^T}},
\end{equation}

여기서 $\otimes$는 벡터의 outer product 입니다. 이 공식을 기본으로, 파란색으로 표현된 부분을 먼저 계산함으로써 attention은 선형화 될수 있습니다. 각 분모, 분자 부분을 이전 계산으로 부터 cumsum이므로 상수시간으로 계산됩니다. 이는 디코더가 RNN처럼 run할수 있게 합니다.

식 (16) 모델은 K-V 외적으로 표현되는 연관성을 집계하여 메모리 행렬을 유지 한 다음 정규화를 통해 통해 특성 매핑된 Q와 메모리 행렬을 곱하여 V를 겁색합니다. 이접근법에는 3가지 주요 구성요소가 있습니다.

####  Feature Maps
Linear Transformer는 $\phi : elu(x) + 1$을 사용합니다. 이는 내적을 근사하는것에 초점을 맞추지는 않았지만, Trm과 비슷한 성능을 경험적으로 입증하였습니다.


Performer는 스코어링 함수로 random feature maps를 사용합니다. random feature map은 아래와 같습니다.

\begin{equation}
\phi(x) = \frac{h(x)}{\sqrt{m}}[f_1(\omega_1^Tx), \cdots, f_m(\omega_1^Tx), \cdots, f_l(\omega_m^Tx)],
\end{equation}

Performer의 첫 번째 버전은 원래 가우시안 커널을 근사화하는 데 사용 된 random Fourier feature map에서 영감을 받았습니다. 이는 $h(x)= exp(\frac{\|x\|^2}{2}),\  l=2,\  f_1=sin,\  f_2 = cos$을 사용합니다.  
trigonometric random feature map이 편향되지 않는 근사치를 생성하지만 attention score가 음수 일수 있기 때문에(음이 아닌 attention score를 보장하지 않음) 불안정한 동작과 비정상적인 동작을 유발할 수 있습니다.  
version 2 에서는 exp를 사용하여 편향을 보장합니다.

#### Aggregation Rule
RFA는 로컬 종속성을 모델링 하기 위해 gating 매커니즘을 도입합니다. 특히 memory matrix S에 새로운 association이 추가될때 특정 시간 단계에서 학습가능한 입력 종속 스칼라 g에 의해 S에 가중치를 부여하고 (1-g)에 의해 새로운 association에 가중치를 부여 합니다. 이를 통해 히스토리 연관은 기하급수적으로 감소하고 각 시간 단계에서 최근 컨텍스트가 선호 됩니ㅏㄷ.

### Query Prototyping and Memory Compression
sparse attention or kernel-based linearized attention말고도 Q, K-V의 수를 줄임으로써 복잡성을 줄일수 있습니다. 이는 각각 query prototyping, memory compression으로 이어집니다.

Fig. 8.

#### Attention with Prototype Queries
여러 프로토 타입이 attention의 분포를 계산하는 주요 소스 역할을 합니다. 모델은 표현된 Q의 위치에 분포를 복사하거나 해당 위치를 불연속 균일 분포로 채웁니다.

Clustered Attention는 Q-클러스터링 -> attention분포 계산 -> 같은 클러스터속 Q는 분포를 공유합니다.
Informer는 명시적 Q희소성 측정을 사용하여 Q에서 프로토 타입을 선택합니다. 이는 Q의 attention 분포와 불연속 균일 분포 사이의 Kullback-Leibler divergence 근사치에서 파생 되었습니다. attention 분포는 Q희소성 측정에서의 상위 u개 쿼리에 대해서만 계산됩니다. 나머지 Q는 불연속 균등 분포로 할당 됩니다.

#### Attention with Compressed Key-Value Memory
Q처럼 K-V쌍의 수를 줄임으로써 복잡성을 줄일 수 있습니다.

Liu et al.는 strided conv를 사용하여 K-V수를 줄이는 Memory Compressed Attention (MCA)를 제안합니다.  
이는 동일한 작업(sec 4.1)에 제안된 local attention에 대한 보완으로 사용되며 global context를 capture할수 있습니다. 이 메커니즘은 커널 크기 k의 요소에 의해 K-V쌍의 수를 줄이므로, 동일한 리소스에서 바닐라 Trm보다 긴 Seq를 처리 할 수 있습니다.

Set Transformer와 Luna는 학습가능한 여러 외부 전역 노드를 사용하여 입력 정보를 요약한 다음, 요약된 표현은 입력이 처리하는 압축 메모리 역할을 합니다. 이는 quadratic복잡성을 선형으로 줄입니다.

Linformer는 linear projection을 사용해서 길이 $n \to n_k$으로 K-V를 투영합니다. 이 또한 선형 복잡성으로 줄입니다.  
단점은 입력 seq 길이를 가정해야 하므로  autoregressive attention에서는 사용 불가입니다.

Poolingformer는 sliding window attention + compressed memory의 결합을 사용합니다. 수용장을 넓히기 위해 sliding window 이후에 compressed module을 사용합니다.

### Low-rank Self-Attention
경험적 및 이론적 분석은 attention matrix가 종종 low rank라고 보고합니다.  
이는 두가지를 나타 냅니다.
1. low-rank 속성을 매개변수화로 명시적인 모델링이 가능
2. A는 low-rank로 근사될 수 있음

####  Low-rank Parameterization
$Rank(A) < T$ 라는 사실은 입력이 짧은 경우($D_k >T)$오버 피팅으로 이어집니다.
따라서 $D_k$를 제한하여 낮은 rank속성을 inductive bias로 명시적으로 모델링 하는것이 합리적입니다.

Guo et al. 은 A를 장거리 non-local interactin을 캡쳐하는 작은 $D_k$와 local 종속성을 캡쳐하는 band attention module로 low-rank attention module로 분해합니다.

####  Low-rank Approximation
A의 또다른 low rank property은 self attention의 복잡성을 줄이기 위해 행렬 근사를 사용할 수 있다는 것 입니다.
관련된 방법론은 커널 행렬의 low-rank근사입니다.

sec. 4.2에서 선형화 attention 방법중 일부는 random feature map을 사용한 커널 슨사에서 영삼을 얻었습니다.  Performer는 원래 가우시안 커널을 근사하기 위해 제안된 random Fourier feature map을 따릅니다.  
이 방법은 먼저 attention 분포 행렬 $A$를 $C_QGC_K$로 근사 합니다. 여기서 $G$는 가우시안 커널 행렬이고 random feature map은 $G$를 근사하는데 사용됩니다.  

또 다른 방법은 Nyström 방법의 아이디어를 따릅니다. 먼저 다운 샘플링 방법()을 사용하여 T입력에서 m 랜드마크 노드를 선택합니다. $\tilde{Q}, \tilde{K}$를 선택된 랜드 마크 Q, K로 설정하면 다음 근사값이 attention 계산에 사용됩니다.

\begin{equation}
\tilde{A} = softmax(Q\tilde{K}^T)(softmax(\tilde{Q} \tilde{K}^T))^{-1}softmax(\tilde{Q}K^T)
\end{equation}

여기서 중간 부분 $M^{-1} = (softmax(\tilde{Q} \tilde{K}^T))^{-1}$는 항상 존재하지 않습니다. 따라서 CSALR는 항등행렬을 추가해주어 역행렬이 존재 하게 만들어 줍니다. 

### Attention with Prior
attention 메커니즘은 일반적으로 가중치가 V에 대한 attention분포인 벡터의 가중 합계로 attended V값을 출력합니다.  
전통적으로 분포는 입력(A)에서 생성됩니다. As a generalized case, attention 분포는 다른 출처에서 올수도 있습니다.  
prior attention 분포는 입력에서 생성된 분포를 보완하거나 대처할 수 있습니다. 아래 그림 처럼 일반화 된 attention에 prior를 가중합 합니다.

Fig. 9.

#### Prior that Models locality
일부 유형의 데이터(ex. text)는 locality에 대한 강한 선호도를 나타낼 수 있습니다. 이는 prior로 명시적으로 인코딩 할 수 있습니다. 간단한 방법은 위치에 대한 가우스 분포를 사용하는 것 입니다. 특히 생성됨 분포에 가우스 density를 곱한다음 재 정규화 할 수도 있습니다. 이는 생성된 A에 bias term G를 추가하는것과 같습니다.

Yang et al. 은 각$q_i$로 부터 $p_i$(중심)을 예측한다음(using a simple feedforward network) 아래 G bias를 정의 합니다.

\begin{equation}
G_{ij} = -\frac{(j - p_i)^2}{2\sigma^2},
\end{equation}

여기서 시그마는 하이퍼 파라미터 또는 입력으로 부터 예측된값입니다.

Gaussian Transformer은 G를 다음과 같이 정의 합니다.

\begin{equation}
G_{ij} = -|w(i-j)^2 + b|,
\end{equation}

여기서 w, b는 각각 편차를 제어하고 중심 위치에 대한 가중치를 줄이는 스칼라 매개 변수입니다.

#### Prior from Lower Modules

Trm 아키텍처에서는 인접 layer에서 attention 분포가 유사하다는 것이 종종 관찰됩니다. 따라서 attention을 계산 하기 위한 prior으로 이전 layer에서 attention 분포를 제공하는 것은 당연합니다. 마지막 attention score는 아래와 같이 정의 됩니다.

\begin{equation}
\hat{A}^{(l)} = w_1 \cdot A^{(l)} + w_2 \cdot g(A^{(l-1)})
\end{equation}

여기서 g는 previous 점수를 prior로 변환하는 함수 입니다.

Predictive Attention Transformer는 이전 attention score에 2D Conv를 적용하고 생성된 점수와 convex 조합으로 내부 점수를 계산하자고 제안했습니다. 이는 $w_1 = \alpha, w_2 = 1 - \alpha, g : conv$입니다.실험에서 사전 훈련됨 모델, 일반 모델 모두 성능이 향상되었습니다.

Realformer는 residual skip connection처럼 직접 추가 하였습니다.($w_1 = w_2 = 1 , g : identity map$)

극단적인 경우 Lazyformer는 인접한 여러 layer간에 attention map을 공유합니다.  
$g : identity map$이며 $w_1 = 0, w_2 = 1$과 $w_1 = 1, w_2 = 0$의 설정을 번갈아 가며 전환하는 것과 같습니다.  
이는 attention map이 한번만 계산되어 계산 비용이 절감 됩니다.

#### Prior as Multi-task Adapters
Adapters는 task에 따라 효율적인 매개변수 공유를 위해 사전 훈련된 네트워크의 특정 위치에 부착되는 작업 종속적인 훈련가능 모듈 입니다.  
Pilault et al. 은 Conditionally Adaptive Multi-Task Learning (CAMTL)을 제안합니다. CAMTL은 task encoding $z_i$에 의존하는 훈련 가능한 attention prior $M(z_i)$를 사용합니다.

\begin{equation}
M(z_i) = \bigoplus_{j=1}^{m}A_j'(z_i),\  A_j'(z_i) = A_j \gamma_i(z_i) + \beta_i(z_i),
\end{equation}

#### Attention with Only Prior.
일부는 입력간 pair-wise interaction과 독립적인 attention 분포를 사용하여 연구했습니다. 즉, 모델이 prior분포만을 사용합니다.

Zhang et al. 은 attention 분포의 유일한 소스로 이산 균일 분포를 사용하는 average attention network라고 하는 효율적인 디코더의 변형을 설계하였습니다. 따라서 모든 value는 모든 값의 누적 평균으로 집계됩니다.

You et al.은 attention 계산을 위해 하드 코딩된 attention 분포로 가우스 분포를 활용합니다. attention 분포는 특정 local window에 초점을 맞춰야 한다는 점에서 Yang et al. and Guo et al.과 비슷합니다. 정확히는 생성된 attention을 완전히 drop하고 attention 계산에 가우스 분포만 사용합니다. 이 접근 법에는 평균(중심 위치) 및 분산은 하이퍼 파라미터가 되도록 설계되었습니다. 기계 번역 task 실험에서 기준 모델과 유사한 성늘을 달성 할 수 있었습니다.

Synthesizer는 생성된 attention score를 (1) 학습 가능하고 무작위로 초기화 된 attention score (2)Q 입력 자체에만 조건화 된 FFNN에 의해 출력된 attention score로 대체 할 것을 제안 합니다. 번역, 언어 모델링 실험에서 바닐라 Trm와 경쟁력 있는 성능을 달성 할 수 있음을 보여주었지만 이러만 변형이 작동하는 이유는 설명되어 있지 않습니다.

### Improved Multi-Head Mechanism
MHA은 다른 위치의 다른 representation subspace에 공동으로 attend할 수 있는 능력이 매력적입니다. 그러나 서로 다른 attention head(이하 AH)가 실제로 뚜렷한 특징을 포착하도록 보장하는 메커니즘은 없습니다. 

#### Head Behavior Modeling
MHA의 기본 동기는 모델이 서로 다른 위치에서 서로 다른 representation subspace의 정보에 공동으로 참석할 수 있도록 하는 것 입니다. 하지만 Trm에서 AH간에 다른 동작을 보장하는 메커니즘이 없으며 head가 서로 상호작용하는 메커니즘도 없습니다. 

이 work line은 다른 AH의 동작을 안내하거나 AH간의 상호 작용을 허용하는 보다 정교한 메커니즘을 통합하여 MHA 메커니즘을 개선합니다. 

Li et al. [73]은 loss function에 auxiliary disagreement regularization term을 도입하여 서로 다른 AH간의 다양성을 장려합니다. regularization term은 두개의 정규화 항으로 구성되어 각각 1. 입력 subset 및 출력 representation의 cos 거리를 최대화 하고 2. 해당 A matrix의 element-wise multiplication을 통해 여러 head가 참석하는 위치를 분산 시킵니다. 

여러 조사에 따르면 사전 훈련됨 Trm은 언어적 지원이 거의 없는 특정 패턴이 self-attention에 나타납니다. 대표적으로 BERT에서 몇가지 간단한 attention 패턴을 식별합니다.(많은 AH는 \[CLS\], \[SEP\]에 단순히 attention합니다) 결과적으로 Trm 모델의 학습을 향상 시키기 위해 몇가지 제약이 도입 될 수 있습니다. 이를 위해 Deshpande and Narasimhan은 attention 분포 map과 미리 정의된 attention 패턴 사이의 Frobenius norm으로 정의되는 auxiliary loss function를 사용할 것을 제안 합니다.

Talking-head Attention는 생성된 attention score을  linearly project하고 해당 공간에 softmax를 적용한 다음 value aggregation을 위해 $h_v$를 project하는 메커니즘을 사용합니다. 이는 모델이 학습가능한 방식으로 AH간에 정보를 이동하도록 장려합니다.

Collaborative Multi-head Attention은 attention간 $W^Q$와 $W^K$를 공유하고 i 번째 헤드에 대해 mixing vector $m_i$를 사용하여 아래 식에 적용합니다. 

\begin{equation}
head_i = Attention(QW^Qdiag(m_i), KW^K, VW_i^V),
\end{equation}

#### Multi-head with Restricted Spans

바닐라 A는 Q가 모든 K-V 쌍에 참여할 수 있는 전체 attention span을 채택합니다. 그러나 일부 head는 주로 local context에 집중하는 반면 다른 head는 더 넓은 context에 집중하는것으로 관촬됩니다. 따라서 attention spans를 제한 하는것이 유익할 수 있습니다. 
+ **Locality**  
attention 범위를 제한하면 명시적인 local 제약이 발생합니다. 이것은 지역성이 중요한 prior인 경우 유리합니다.
+ **Efficiency**  
적절히 구현하면 추가 메모리, 계산 비용 없이 매우 긴 seq로 확장가능합니다.

attention 범위를 제한 하는것은 각 attention분포 값에 마스크 값을 곱한 다음 다시 정규화 하는 것으로 표현 할 수 있습니ㅏㄷ. 여기서 마스크는 \[0. 1\]의 값에 거리를 매핑하는 non 증가 함수로 표현가능 합니다. 바닐라 attention은 아래 그림에서 (a)처럼 모든 거리에 마스크 1을 할당하는것과 같습니다.

Fig. 10.

Sukhbaatar는 학습가능한(scalar z) attention span을 제안합니다. 이는 모델이 feature의 계층적 구성을 학습할수 있게 합니다. 또한 더 적은 연산수로 더 좋은 성과를 냈습니다.

Multi-Scale Transformer는 10 - (c)처럼 w에 의해 제어되는 고정 window내에서 attention이 제한 됩니다. BERT 관찰 결과로 상위 레이어는 더 큰 범위, 하위 레이어는 더 작은 스케일로 제한하여, 실험에서 긴 seq에 대한 추론을 가속화 하면서 성능을 올릴수 있음을 보였습니다.

#### Multi-head with Refined Aggregation
바닐라 MHA의 concatenate-and-project formulation의 projection은 아래와 같이 분할될수 있습니다.

\begin{equation}
  W^O = [W_1^O;W_2^O;\cdots;W_H^O]
\end{equation}

즉, 다음과 같이 표현됩니다.

\begin{equation}
  MHA(Q, K, V) = \sum_{i=1}^{H}Attention(QW_i^Q, KW_i^K, V\color{Blue}{W_i^VW_i^O})
\end{equation}

이 식을 보면 MHA의 표현력을 완전히 활용하지 않아서 이전(전체)가 더 바람직 하다고 할 수 도 있습니다.

#### Other Modifications
MH 메커니즘을 향상시키기 위한 기타 방법입니다.

Shazeer [116]는 K-V상이 AH간에 공유되는 다중 쿼리 attention을 제안합니다. 메모리가 적고, 디코딩 속도가 높지만 약간의 성능저하가 있습니다.

Bhojanapalli et al. [11]은 small attention key size가 attention 분포를 나타내는 능력에 영향을 미칠 수 있음을 확인 하였습니다. 이에 $D_m/h$ size 대신 $h$로 할 것을 제안합니다.

##  OTHER MODULE-LEVEL MODIFICATIONS

### Position Representations
**Definition 5.1** (permutation equivariant function). Let $\Pi_n$ be the set of all permutations of indices $\{1, 2, · · · ,T \}$. A function $f : X^T \to Y^T$ is said to be permutation equivariant if and only if for any $\pi \in \Pi_T$

\begin{equation}
  f(\pi x) = \pi f(x)
\end{equation}

Conv와 recurrent는 permutation equivariant하지 않습니다. attention module, position-wise FFNN는 permutation equivariant합니다. 즉, 입력 구조가 필요한 설정은 입력 문제 이외에 모델링에 관련해서 문제가 될 수 있습니다. 
예를들어 text seq는 순서가 중요하므로 Trm 아키텍처에서 단어의 위치를 올바르게 인코딩하는것이 중요합니다. 따라서 Trm에 위치 정보를 주입하려면 추가 메커니즘이 필요합니다. 일반적인 설계는 벡터를 사용하여 위치 정보를 표현한 다음 벡터를 추가 입력으로 모델에 주입하는 것 입니다.

####  Absolute Position Representations
바닐라 Trm에서 위치 정보는 position encoding으로 encode됩니다. 각 위치 인덱스 $t$에서 인코딩 벡터 $p_t = PE(t) \in \mathbb{R}^{D_m}$이고 각 원소는 삼각 함수 값입니다. 

\begin{equation}
PE(t)_i =
\begin{cases}
sin(\omega_i t), & \mbox{if }i\mbox{ is even} \\
cos(\omega_i t), & \mbox{if }i\mbox{ is odd}
\end{cases}
\end{equation}

여기서 omega는 hand-crafted frequency입니다.

절대 위치를 나타내는 또 다른 방법은 각 위치에 대한 위치 임베딩 세트를 학습하는 것입니다. 이는 hand-crafted position embedding에 비해 더 유연하지만 max leght 때문에 훈련 때 보다 긴 seq는 처리 불가능합니다.

Wang et al. [138]는 sinusoidal position representation을 제안하지만 각 주파수를 데이터에서 학습합니다. 이는 유도성을 유지하면서 유연합니다.

FLOATER [85]는 position representation을 연속적인 동적 시스템으로 프레임화 하는 Neural ODE(상 미분 방적식)를 체텍하여 back prop으로 학습을 가능하게 합니다. 이는 귀납적이고 유연하며 fully learnable approch보다 효율적입니다.

바닐라 접근법은 토큰의 위치 인코딩을 추가 하지만 입력 신호가 layer를 통과 함에 따라 상위 레이어에서 위치 정보가 손실 될 수 있습니다. 즉, 나중에 각 Trm layer의 입력에 위치 표현을 추가 하는것이 유익한 작업을 수행합니다.

#### Relative Position Representations
개별 토큰의 위치 대신 토큰간의 위치 관계를 나타내는데 중점을 둡니다. 
직관은 self-attention에서 입력 elements(방향, 거리)간의 pair-wise-positional 관계가 elements의 위치보다 더 좋다는 것 입니다. 이 원칙을 따르는 방법을 Relative Position Representations이라고 합니다.

Shaw et al. [115]는 attention 메커니즘의 K에 학습가능한 Relative Position Embedding을 추가하자고 제안합니다.

\begin{equation}
k_j' = k_j + r_{ij}, \  for i = 1, \cdots, n,
\end{equation}

\begin{equation}
r_{ij} = R_{clip(i-j)},
\end{equation}

\begin{equation}
clip(x) = max(-K, min(x, K)),
\end{equation}

여기서 $r_{ij}$는 위치 i와 j사이의 관계를 위한 Relative Position Embedding입니다. K는 embedding의 수를 결정하는 가장 큰 오프셋입니다. 일반적으로 K는 대부분의 입력을 수용할 수 있는 길이로 설정합니다.

Transformer-XL [24] sin 인코딩을 사용하여 위치 관계를 나타내지만 attention scoer 계산을 재설계하여 내용과 위치 정보를 아래 식과 같이 융합합니다.

\begin{equation}
A_{ij} = q_ik_j^\intercal + q_i\left( R_{i-j}W^{K,R}\right)^\intercal + u_1k_j^\intercal + u_2\left( R_{i-j}W^{K,R}\right)^\intercal
\end{equation}

여기서 $W, u_1, u_2$는 학습가능한 파라미터 이고 $R$은  sinusoidal encoding matrix입니다.  
이후 scoer A에 softmax를 계산하여 attention값을 구합니다.   
Note that) the learnable sinusoidal encoding is also a drop-in replacement to hand-crafted R.

DeBERTa [50]는 Shaw[115] 같은(like) 위치 임베딩을 사용합니다. 그리고 Transformer-XL 처럼 분리된 임베딩 스타일을 사용합니다.

\begin{equation}
A_{ij} = q_ik_j^\intercal + q_i\left( r_{ij}W^{K,R}\right)^\intercal + k_j^\intercal\left( r_{ij}W^{Q,R}\right)^\intercal
\end{equation}

여기서 W는 학습 가능한 파라미터 이고 r은 학습가능한 relative positional embeddin입니다($r_{ij} = R_{clip(i-j)}$)
첫번째 term은 content-to-content attention으로 해석 되고 이후 term은 각각 (상대적) content-to-position, position-to-content attention으로 해석됩니다.

#### Other Representation
절대 + 상대적 위치 정보를 모두 포함하는 하이브리드 위치 표현을 연구합니다. Transformer with Untied Position Encoding (TUPE)[63]는 content-to-content term, absolute position-to-position term, relative positional relationships을 표현하는 bias term으로 A를 계산합니다.

\begin{equation}
A_{ij} = q_ik_j^\intercal + \left(q_iW^{Q,P}\right) \left(q_jW^{K,P}\right)^\intercal + b_{j-i}
\end{equation}

absolute와 relative 정보를 모두 표현하는 single position representation을 설계할 수 도 있습니다.  
Roformer [123]는 RoPE(Rotary Position Embedding)를 사용하여 $t$번째 입력 $x_t$의 아핀 affine-transformed embedding에 회전 행력 R을 곱하여 토큰의 위치를 나타냅니다.


\begin{equation}
q_t = x_tW^QR_{\Theta, t},\  k_t = x_tW^KR_{\Theta, t},
\end{equation}


\begin{equation}
R_{\Theta, t} = \bigoplus_{j=1}^{D_k/2}M(t, \theta_j),
\end{equation}

여기서 $M$은 2D 회전 변환입니다. 이 방법의 장점은 유도된 representation이 변하지 않는다는 것입니다.  
즉, q, k의 attention score는 상대적 위치 오프셋과 만이 관련됩니다.

\begin{equation}
q_ik_j^\intercal = \left(x_iW^Q\right)R_{\Theta, \color{Blue}{j-i}} \left(x_jW^K\right)^\intercal
\end{equation}

실제로 임베딩 행렬 곱셈은 memory footprint를 줄이기 위해 두가지 element-wise multiplication으로 구현할 수 있습니다. RoPE는 absolute embedding을 사용하지만 relative positional relations를 포착할 수 있습니다. 이 접근 방식은 Sec. 4.2.에 설명된 linearized attention과 호완될수 있습니다.

#### Position Representations without Explicit Encoding
위치 인코딩을 명시적으로 도입하는 대신 위치에 대한 연속 함수로 임베딩을 일반화 하여 단어 임베딩에서 위치 정보를 인코딩 하는 방법을 제안합니다.

R-Transformer [143]는 local RNN을 사용하여 순차 데이터의 지역성을 모델링합니다. 특히 R-transformer의 각 블록에 대한 입력은 먼저 local RNN에 들어간 다음 MHA에 들어갑니다. RNN 구조는 순서 정보를 도입하고 self-attention을 보완하기 위해 로컬 종속성을 캡쳐 합니다. 

Conditional positional encoding (CPE) [20]는 제로 패딩을 사용한 conv를 사용하여 Vision Transformer(ViT)에 대한 각 레이어에서 조건부 위치 인코딩을 생성합니다. 이는 conv의 제로 패딩이 절대 위치 정보를 암시적으로 인코딩 할 수 있다는 직관에서 옵니다.

#### Position Representation on Transformer Decoders

디코더의 마스킹된 self-attention이 순열 등변성이 아닌것은 중요한 정보입니다. 따라서 Trm의 디코더만을 이용하는 모델은 명시적인 위치 표현을 통합하지 않고도 위치 정보를 감지 할 수 있습니다. 이것은 언어 모델링 작업에 대한 일부 결과[59, 112]이며, 저자들은 위치 인코딩을 제거 하면 성능이 향상될 수 하였습니다.

###  Layer Normalization
Layer Normalization(이하 LN)은 훈련을 안정화 하는 메커니즘입니다. LN모듈을 분석하고 개선하기 위한 몇가지 작업을 서술합니다.

####  Placement of Layer Normalization
바닐라 Trm에서 LN은 residual block사이에 있습니다. 이를 post-LN이라고 합니다. Later Transformer implementations [67, 135]은 아래 그림과 같이 LN의 순서를 바꿉니다.(pre-LN)

Xiong et al. [150]은 이론적으로 Trm으 ㅣ기울기를 조사하여 출력 레이어 근처의 기울기가 post-LN Trm에서 초기에 크다는 것을 발견했습니다. 이것은 warm-up이 없는 post-LN Trm이 불안정한 훈련의 원인이 될수 있다고 합니다.
반면 pre-LN Trm은 동알한 문제를 격지 않았습니다. 따라서 Pre-LN Trm에 대해 Warm-up stage를 안전하게 제거 할 수 있음을 추론하고 실험을 통해 검증하였습니다.